In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# The URL of the website, with the page number included
url = 'https://maharera.maharashtra.gov.in/projects-search-result?page={}'

# Total number of results
results = 400

# Deriving the number of pages you want to scrape
numOfPages = ( results + 9 ) // 10

In [ ]:
# Define a function that will handle the scraping of one page
async def scrape_page(session, page_num):
    # In your scrape_page function
    async with session.get(url.format(page_num), timeout=200000) as response:
        html = await response.text()


    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all divs
    divs = soup.find_all('div', class_='rounded')

    # Initialize an empty list to store the data
    data = []

    # Loop through each div
    for div in divs:
        # Find the link to redirect to the "View details" page in the div
        link = div.find('a', class_='click-projectmodal')

        if link is not None:  # Make sure a link was found
            href = link['href']

            # Navigate to the page
            async with session.get(href) as response:
                html = await response.text()

            # Parse the HTML content of the page with BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Extract the information you want from the new page
            infos = soup.find_all('div', class_='form-group')

            # Create a dictionary to store this row of data
            row = {}

            # Loop through each info
            for info in infos:
                # Find all divs within the info
                divs = info.find_all('div')

                if(len(divs) % 2 == 1):
                    # Loop through each pair of divs (key and value)
                    for i in range(0, len(divs) - 2, 2):
                        # Get the key from the label in the first div and value from the second div
                        
                        if 'IndOccu' in divs[i].get('id', ''):
                            divs1 = divs[i].find_all('div')
                            key = divs1[0].text.strip()
                            value = divs1[1].text.strip()
                            row[key] = value
                            break
                            
                        if 'clearfix' in divs[i].get('class', []) and  i == 0:
                            key = divs[1].text.strip()
                            value = divs[2].text.strip()
                            row[key] = value
                            break
                            
                        key = divs[i].text.strip()
                        value = divs[i + 1].text.strip()

                        # Add the key-value pair to the dictionary
                        if (key.endswith(":") or value.endswith(":")):
                            continue
                        else:
                            row[key] = value

                else:
                    for i in range(0, len(divs) - 1, 2):
                        # Get the key from the label in the first div and value from the second div
                        
                        if 'IndOccu' in divs[i].get('id', ''):
                            divs1 = divs[i].find_all('div')
                            key = divs1[0].text.strip()
                            value = divs1[1].text.strip()
                            row[key] = value
                            break
                            
                        if 'clearfix' in divs[i].get('class', []) and  i == 0:
                            key = divs[1].text.strip()
                            value = divs[2].text.strip()
                            row[key] = value
                            break
                            
                        key = divs[i].text.strip()
                        value = divs[i + 1].text.strip()

                        # Add the key-value pair to the dictionary
                        if (key.endswith(":") or value.endswith(":")):
                            continue
                        else:
                            row[key] = value

            # Add the dictionary to the list
            data.append(row)

    # # Add a delay before the next request
    # await asyncio.sleep(1)  # delay for 1 second

    return data

In [ ]:
async def main():
    async with aiohttp.ClientSession(trust_env=True) as session:
        tasks = [scrape_page(session, i) for i in range(1, numOfPages)]
        data = await asyncio.gather(*tasks)

    # Flatten the results into a single list
    data = [item for sublist in data for item in sublist]

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)

    nan_value = float("NaN") 
    df.replace("", nan_value, inplace=True) 
    df.dropna(how='all', axis=1, inplace=True) 

    # Write the DataFrame to an Excel file
    df.to_excel('output3.xlsx', index=False)

In [ ]:
# Run the main function
await main()